### Import basic necessary libraries

In [8]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import csv

### Sorting images from .csv file

In [2]:
'''
names = data['NAME'].unique()
images_path = ("..\Datasets\mpii_human_pose\human_pose_images")
count = 0
for name in names:
    source_path = images_path + "\\" + name
    category = np.array(data.loc[data['NAME'] == name, 'Category'])[0]
    destination_path = images_path + "\\" + "in_csv" + "\\" + name
    
    if os.path.isfile(source_path):
        os.rename(source_path, destination_path)
    count += 1
    if count % 250 == 0:
        print(count)

print(count)
''';

In [3]:
data = pd.read_csv('mpii_dataset_clean_v2.csv')
data

,NAME,Activity,Category
0,015601864.jpg,curling,sports
1,015599452.jpg,curling,sports
2,005808361.jpg,curling,sports
3,086617615.jpg,curling,sports
4,060111501.jpg,curling,sports
...,...,...,...
17367,033474347.jpg,pushing car,transportation
17368,082650067.jpg,pushing car,transportation
17369,072772110.jpg,pushing car,transportation
17370,039361034.jpg,pushing car,transportation


In [4]:
data.Category.unique()

array(['sports', 'inactivity quiet-light', 'miscellaneous', 'occupation',
       'water activities', 'home activities', 'lawn and garden',
       'religious activities', 'winter activities',
       'conditioning exercise', 'bicycling', 'fishing and hunting',
       'dancing', 'walking', 'running', 'self care', 'home repair',
       'music playing', 'transportation', 'volunteer activities'],
      dtype=object)

In [2]:
'''print(data.isnull().values.any())
print(data['Activity'].unique().shape)
print(data['Category'].unique().shape)
print(data['Activity'].unique())''';

### Image rescaling script (if necessary)

In [6]:
'''
from PIL import Image
import os, sys

path = ('..\Datasets\mpii_human_pose\human_pose_images')
dirs = os.listdir(path)
outpath  = (r'..\Datasets\mpii_human_pose\resized_images' + '\\')
count = 0
for item in dirs:
    count += 1
    obj = path + '\\' + item
    if os.path.isfile(obj):
        im = Image.open(obj)
        f, e = os.path.splitext(item)
        imResize = im.resize((256, 256), Image.ANTIALIAS)
        imResize.save(outpath + '\\' + f + '.jpeg', 'JPEG', quality=90)
        if count % 100 == 0:
            print(count)
            ''';

### Sorting images in corresponding folders.

In [7]:
x = data.drop('Category', axis=1)
y = data['Category']

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x, y, 
                                                 test_size = 0.20, random_state = 2)
x_train

,NAME,Activity
230,069597421.jpg,"sitting, talking in person, on the phone, comp..."
2141,073872315.jpg,"standing, arts and crafts, sand painting, carv..."
15789,058772539.jpg,"eating, sitting"
2386,069963957.jpg,"rowing, stationary"
10744,061143424.jpg,"bicycling, stationary"
...,...,...
14696,006110708.jpg,ballroom
1099,039090600.jpg,"irrigation channels, opening and closing ports"
11798,073432341.jpg,"water aerobics, water calisthenics"
6637,093874989.jpg,soccer


**Sorting in 'train' and 'test' folders. Only to be done once since seed value random_state=2 will always output the same distribution**


In [9]:
'''
names_train = x_train['NAME'].unique()
names_test = x_test['NAME'].unique()

names = np.concatenate((names_train, names_test), axis=0)

images_path = ("..\Datasets\mpii_human_pose\human_pose_images")
count = 0
for name in names:
    source_path = images_path + "\\" + name
    if name in names_train:
        folder = 'train'
    elif name in names_test:
        folder = 'test'
        
    destination_path = images_path + "\\" + folder + "\\" + name
    
    if os.path.isfile(source_path):
        os.rename(source_path, destination_path)
    count += 1
    if count % 250 == 0:
        print(count)

print(count)
''';

**Sorting images in the different (20) category folders**

In [10]:
'''
# Repartir imágenes en carpetas-categorías

names_train = x_train['NAME'].unique()
names_test = x_test['NAME'].unique()

names = np.concatenate((names_train, names_test), axis=0)

images_path = ("..\Datasets\mpii_human_pose\human_pose_images")
train_dir = os.path.join(images_path, "train")
test_dir =  os.path.join(images_path, "test")

categories = np.array(data['Category'].unique())

# Crear carpetas para categorías
#for category in categories:
#    os.mkdir(os.path.join(train_dir, category))
#    os.mkdir(os.path.join(test_dir, category))

count = 0
for name in names:
    category_image = np.array(data[data['NAME'] == name].Category)[0]

    if name in names_train:
        folder = 'train'
    elif name in names_test:
        folder = 'test'      
        
    source_path = os.path.join(images_path, folder, name)
    destination_path = os.path.join(images_path, folder, category_image, name)

    if os.path.isfile(source_path):
        os.rename(source_path, destination_path)
        
    count += 1
    if count % 250 == 0:
        print(count)
''';

# Convolutional Neural Network (CNN) Model

In [1]:
import tensorflow as tf

In [2]:
model = tf.keras.models.Sequential([
    # Desired size of the image 256x256 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten layer
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Output layer
    tf.keras.layers.Dense(20, activation='softmax')    
])

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 41472)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               21234176  
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1

In [22]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics=['accuracy'])

In [9]:
images_path = ("..\Datasets\mpii_human_pose\human_pose_images")
train_dir = os.path.join(images_path, "train")
test_dir =  os.path.join(images_path, "test")

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1.0/255.0)

test_datagen = ImageDataGenerator(rescale = 1.0/255.0)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = 20,
                                                    class_mode = 'categorical', 
                                                    target_size =(150,150))

test_generator = test_datagen.flow_from_directory(test_dir, batch_size = 20,
                                                 class_mode = 'categorical', 
                                                  target_size = (150,150))

Found 13897 images belonging to 20 classes.
Found 3475 images belonging to 20 classes.


In [25]:
history = model.fit(train_generator, validation_data = test_generator,
                    #steps_per_epoch = 1200, 
                    epochs = 100, # epochs a 100
                    #validation_steps = 50, 
                    verbose = 1)

Epoch 1/100
695/695 [==============================] - 783s 1s/step - loss: 2.3192 - accuracy: 0.3014 - val_loss: 2.0774 - val_accuracy: 0.3672
Epoch 2/100
695/695 [==============================] - 540s 777ms/step - loss: 1.4513 - accuracy: 0.5588 - val_loss: 1.8508 - val_accuracy: 0.4696
Epoch 3/100
695/695 [==============================] - 501s 721ms/step - loss: 0.5329 - accuracy: 0.8403 - val_loss: 2.2403 - val_accuracy: 0.4846
Epoch 4/100
695/695 [==============================] - 518s 745ms/step - loss: 0.1348 - accuracy: 0.9657 - val_loss: 2.9873 - val_accuracy: 0.4668
Epoch 5/100
695/695 [==============================] - 503s 723ms/step - loss: 0.0596 - accuracy: 0.9876 - val_loss: 3.0790 - val_accuracy: 0.4691
Epoch 6/100
695/695 [==============================] - 516s 743ms/step - loss: 0.0608 - accuracy: 0.9847 - val_loss: 3.7640 - val_accuracy: 0.4581
Epoch 7/100
695/695 [==============================] - 510s 734ms/step - loss: 0.0505 - accuracy: 0.9852 - val_loss: 3.54

In [58]:
model.save(r"saved\raw_CNN.h5")

### Testing save and load model

In [11]:
model_from_file = tf.keras.models.load_model('saved/primera_CNN.h5')

In [ ]:
model_from_file.evaluate(test_generator)

133/174 [=====================>........] - ETA: 23s - loss: 1.6322 - accuracy: 0.5060

### Plotting training and validation metrics

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(100)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Draft - Model evaluation with random image### Evaluar modelo con imagen random

In [3]:
'''from tensorflow.keras.preprocessing import image

imgpath = 'fut1.jpg'
imagen = image.load_img(imgpath, target_size=(256,256))

w = image.img_to_array(imagen)
w = np.expand_dims(x, axis=0)   # Mirar documentación de la función
w = np.expand_dims(x, axis=0)
imagenes = np.vstack([w])

classes = model.predict(w, batch_size=20)

print(classes[0])''';